# Uniswap Overview Data

In [2]:
from IPython.core.interactiveshell import InteractiveShell
import warnings
import glob
import os

import pandas as pd
from tqdm import tqdm
import jupyter_black
from ast import literal_eval

from environ.constants import (
    SAMPLE_PERIOD,
    UNISWAP_V2_DATA_PATH,
    UNISWAP_V3_DATA_PATH,
    BETWEENNESS_DATA_PATH,
)

jupyter_black.load()
InteractiveShell.ast_node_interactivity = "all"
warnings.filterwarnings("ignore")

## Unqiue Pair Count

In [4]:
for dex_path, idx in [
    (UNISWAP_V2_DATA_PATH, "v2"),
    (UNISWAP_V3_DATA_PATH, "v3"),
]:
    # load in all folders in constants.UNISWAP_V2_DATA_PATH
    folder_lst = glob.glob(str(dex_path / "swap" / "*"))

    # load in all files in each folder
    file_lst = []
    for folder in folder_lst:
        file_lst.extend(glob.glob(os.path.join(folder, "*.csv")))

    # only keep files that are in the sample period
    file_lst = [
        file
        for file in file_lst
        if (
            SAMPLE_PERIOD[0]
            <= file.split("/")[-1].split("_")[-1].split(".")[0]
            <= SAMPLE_PERIOD[1]
        )
    ]

    # Iterate through all files and get the unique list of pool, token0_id, token1_id
    unique_pool_lst = []
    for file in file_lst:
        df = pd.read_csv(file, usecols=["pool", "token0_id", "token1_id"])
        # drop duplicates
        df = df.drop_duplicates()
        # append to the list
        unique_pool_lst.append(df)

    # concat all the dataframes
    unique_pool_df = pd.concat(unique_pool_lst, ignore_index=True)
    # drop duplicates
    unique_pool_df = unique_pool_df.drop_duplicates()

    # print out the number of unique pools
    print(f"Number of unique pairs in {idx}: {unique_pool_df.shape[0]}")

Number of unique pairs in v2: 122458
Number of unique pairs in v3: 8436


## Total swap number (Atomic)

In [2]:
for dex_path, idx in [
    (UNISWAP_V2_DATA_PATH, "v2"),
    (UNISWAP_V3_DATA_PATH, "v3"),
]:
    # load in all folders in constants.UNISWAP_V2_DATA_PATH
    folder_lst = glob.glob(str(dex_path / "swap" / "*"))

    # load in all files in each folder
    file_lst = []
    for folder in folder_lst:
        file_lst.extend(glob.glob(os.path.join(folder, "*.csv")))

    # only keep files that are in the sample period
    file_lst = [
        file
        for file in file_lst
        if (
            SAMPLE_PERIOD[0]
            <= file.split("/")[-1].split("_")[-1].split(".")[0]
            <= SAMPLE_PERIOD[1]
        )
    ]

    # Iterate through all files and get the number of swaps
    swap_count = 0
    for file in tqdm(file_lst, desc="Counting swaps"):
        swap_count += pd.read_csv(file).shape[0]

    print(f"Total number of swaps of {idx}: {swap_count}")

Counting swaps: 100%|██████████| 975/975 [05:42<00:00,  2.85it/s]


Total number of swaps of v2: 92649419


Counting swaps: 100%|██████████| 610/610 [01:41<00:00,  5.98it/s]

Total number of swaps of v3: 23613266


# Betweenness Trtades

In [4]:
for dex in ["v2", "v3"]:
    # list of all files in the betweenness folder
    file_lst = glob.glob(str(BETWEENNESS_DATA_PATH / "swap_route" / "*"))

    # only keep files that are in the sample period
    file_lst = [
        file
        for file in file_lst
        if (
            (
                SAMPLE_PERIOD[0]
                <= file.split("/")[-1].split("_")[-1].split(".")[0]
                <= SAMPLE_PERIOD[1]
            )
            & (file.split("/")[-1].split("_")[-2] == dex)
        )
    ]

    # Iterate through all files and get the number of transactions
    tx_count = 0
    for file in tqdm(file_lst, desc="Counting transactions"):
        # load in the dataframe
        df = pd.read_csv(file)

        # remove the row with label as Error
        df = df[df["intermediary"] != "Error"]

        # convert the intermediary column to a list
        df["intermediary"] = df["intermediary"].apply(literal_eval)

        # sum up the length of the intermediary column
        tx_count += df["intermediary"].apply(len).sum()

    print(f"Total number of betwenness transactions of {dex}: {tx_count}")

Counting transactions: 100%|██████████| 944/944 [01:54<00:00,  8.24it/s]


Total number of betwenness transactions of v2: 2121236


Counting transactions: 100%|██████████| 607/607 [01:01<00:00,  9.85it/s]

Total number of betwenness transactions of v3: 1180697


In [6]:
# Total number of betweenness swap
2121236 + 1180697

# Percentage of swaps that are betweenness in terms of number of swaps
(2121236 + 1180697) / (92649419 + 23613266)

3301933

0.028400625703767293

# Total Transactions (Full-length)

In [9]:
for dex in ["v2", "v3"]:
    # list of all files in the betweenness folder
    file_lst = glob.glob(str(BETWEENNESS_DATA_PATH / "swap_route" / "*"))

    # only keep files that are in the sample period
    file_lst = [
        file
        for file in file_lst
        if (
            (
                SAMPLE_PERIOD[0]
                <= file.split("/")[-1].split("_")[-1].split(".")[0]
                <= SAMPLE_PERIOD[1]
            )
            & (file.split("/")[-1].split("_")[-2] == dex)
        )
    ]

    # Iterate through all files and get the number of transactions
    tx_count = 0
    for file in tqdm(file_lst, desc="Counting transactions"):
        tx_count += pd.read_csv(file).shape[0]

    print(f"Total number of transactions of {dex}: {tx_count}")

Counting transactions: 100%|██████████| 944/944 [00:33<00:00, 28.30it/s]


Total number of transactions of v2: 18635768


Counting transactions: 100%|██████████| 607/607 [00:18<00:00, 32.74it/s]

Total number of transactions of v3: 9973144


## Top 50 pools: Swap, mint, burn number

In [15]:
for dex_path, idx in [
    (UNISWAP_V2_DATA_PATH, "v2"),
    (UNISWAP_V3_DATA_PATH, "v3"),
]:
    # list of all files in the directional_volume folder
    file_lst = glob.glob(str(dex_path / "directional_volume" / "*"))

    # only keep files that are in the sample period
    file_lst = [
        file
        for file in file_lst
        if (
            SAMPLE_PERIOD[0]
            <= file.split("/")[-1].split("_")[-1].split(".")[0]
            <= SAMPLE_PERIOD[1]
        )
    ]

    # Iterate through all files and get the number of transactions
    mintsCount = 0
    burnsCount = 0
    swapsCount = 0
    for file in file_lst:
        # load in the file
        df = pd.read_csv(file)

        # sum up the number of mints, burns, and swaps
        mintsCount += df["mintsCount"].sum()
        burnsCount += df["burnsCount"].sum()
        swapsCount += df["swapsCount"].sum()

    print(f"Total number of mints of {idx}: {mintsCount}")
    print(f"Total number of burns of {idx}: {burnsCount}")
    print(f"Total number of swaps of {idx}: {swapsCount}")

Total number of mints of v2: 515288.0
Total number of burns of v2: 309294.0
Total number of swaps of v2: 29413825.0
Total number of mints of v3: 361641.0
Total number of burns of v3: 403640.0
Total number of swaps of v3: 13686873.0
